In [1]:
# notebook to illustrate document similarity clusters

In [2]:
# following only used for development, reloads the modules with any code changes
%load_ext autoreload
%autoreload 2

# inline matplotlib charts
%matplotlib inline

In [3]:
# import our text mining toolkit
import text_mining_toolkit as tmt

In [4]:
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/recipes/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(directory_of_files="data_sets/mystery_corpus_01/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/iraq_inquiry/txt/", text_filename_pattern="the-report*.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/clinton_emails/txt/", text_filename_pattern="C0*.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/shakespeare_macbeth/txt/", text_filename_pattern="macbeth_act_0?_scene_0?.txt")
cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/mixed/txt/", text_filename_pattern="*.txt")

content_directory =  data_sets/mixed/txt/
text_filename_pattern =  *.txt
self.documents populated =  52


In [6]:
# first clear index
tmt.index_wordcount.delete_index(cr.content_directory)

# for all documents in corpus
for document_name in cr.get_documents():
    print("processing ", document_name)

    # get document text
    document_text = cr.get_text_by_document(document_name)

    # simplify whitespace (remove newlines)
    b = tmt.text_processing.simplify_whitespace(document_text)

    # only keep alphanumeric characters, removes punctuation
    c = tmt.text_processing.keep_only_alphanumeric(b)

    # make lowercase
    d = tmt.text_processing.to_lowercase(c)

    # split into words list
    dl = tmt.text_processing.split_text_into_words(d)
    
    # build n-grams
    #gl = tmt.word_processing.build_ngrams_from_words(dl,2)

    # remove stop words
    #el = tmt.word_processing.remove_stop_words(dl, "./stopwords/minimal-stop.txt")
    
    # update index
    tmt.index_wordcount.create_wordcount_index_for_document(cr.content_directory, document_name, dl)
    pass



processing  01.txt
processing  02.txt
processing  03.txt
processing  04.txt
processing  05.txt
processing  06.txt
processing  07.txt
processing  08.txt
processing  09.txt
processing  10.txt
processing  11.txt
processing  12.txt
processing  13.txt
processing  C05739545.txt
processing  C05739546.txt
processing  C05739547.txt
processing  C05739550.txt
processing  C05739554.txt
processing  C05739559.txt
processing  C05739560.txt
processing  C05739561.txt
processing  C05739562.txt
processing  C05739563.txt
processing  C05739564.txt
processing  C05739565.txt
processing  C05739567.txt
processing  macbeth_act_01_scene_01.txt
processing  macbeth_act_01_scene_02.txt
processing  macbeth_act_01_scene_03.txt
processing  macbeth_act_01_scene_04.txt
processing  macbeth_act_01_scene_05.txt
processing  macbeth_act_01_scene_06.txt
processing  macbeth_act_01_scene_07.txt
processing  macbeth_act_02_scene_01.txt
processing  macbeth_act_02_scene_02.txt
processing  macbeth_act_02_scene_03.txt
processing  mac

In [7]:
# merge document indices into a corpus index
tmt.index_wordcount.merge_wordcount_indices_for_corpus(cr.content_directory)

saving corpus word count index ...  data_sets/mixed/txt/index_wordcount.hdf5


In [10]:
tmt.index_wordcount.print_index(cr.content_directory)

wordcount_index_file  data_sets/mixed/txt/index_wordcount.hdf5
            01.txt  02.txt  \
               0.0     0.0   
0              0.0     0.0   
0100           0.0     0.0   
0253           0.0     0.0   
03             0.0     0.0   
030311docx     0.0     0.0   
0330           0.0     0.0   
036            0.0     0.0   
04002          0.0     0.0   
05             0.0     0.0   

                                   ...                          \
                                   ...                           
0                                  ...                           
0100                               ...                           
0253                               ...                           
03                                 ...                           
030311docx                         ...                           
0330                               ...                           
036                                ...                           
04002       

In [11]:
tmt.index_wordcount.search_wordcount_index(cr.content_directory, "bread milk")

08.txt    3.0
03.txt    3.0
12.txt    2.0
06.txt    1.0
02.txt    1.0
01.txt    1.0
dtype: float64

In [ ]:
# calculate document similarities
tmt.